## **Environment Setup**
Run all when initiating session## **Environment Setup**
Run all when initiating session

In [ ]:
# retrieve config file, if using colab (nothing should happen if you use Windows)
!cp './drive/My Drive/Live Workspace/generative-facial-cosmetics/model_engineering/config.py' '.'
!mkdir data
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/model_engineering/data/' '.'
!mkdir models
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/model_engineering/models/' '.'
!mkdir technical
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/model_engineering/technical/' '.'
!mkdir trainers
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/model_engineering/trainers/' '.'
!mkdir utils
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/model_engineering/utils/' '.'

In [ ]:
from config import *                            # config.py
from matplotlib import pyplot as plt 
import os
%matplotlib inline

## **GCS Integration**
Run all when initiating session

In [ ]:
if isWindows():
  import utils.gcs_windows as gcs 
elif isColab():
  import utils.gcs_colab as gcs
else:
  raise NotImplementedError('OS is not supported yet')

gcs.init()

BUCKET1 = gcs.mount_bucket("ffhq-1024-lips-1")

## **Model Environment Setup**

In [ ]:
from technical.accelerators import strategy
from data.pipeline import *
from trainers.stylegan import train, LATENT_SIZE, IMAGE_SHAPE
from models.discriminator import build_model as build_discriminator
from models.generator import build_model as build_generator

In [ ]:
training_dataset = get_dataset(BUCKET1, True, True)
generator = build_generator(strategy)
discriminator = build_discriminator(strategy)

## **Pre-Train Testing**

In [ ]:
# generate from untrained generator (should get noises)
noise = tf.random.normal(LATENT_SIZE)
image = generator(noise, training=False)
plt.imshow(image)
print(discriminator(image, training=False).numpy())

## **Training**

In [ ]:
EPOCHS = 200

train(generator, discriminator, training_dataset, EPOCHS, BATCH_SIZE, strategy)

## **Testing**

In [ ]:
noise = tf.random.normal(LATENT_SIZE)
image = generator(noise, training=False)
plt.imshow(image)